In [808]:
# Santiago Rodriguez Murialdo - A01704109
# Actividad M1
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import numpy as np
import time
import random

In [809]:
class CleaningAgent(Agent):
    # Constructor del Agente
    def __init__(self, id, model):
        super().__init__(id, model)
    
    # Funcion para mover el agente basado en su entorno
    def move(self):
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False) # Ver vecinos
        open_cells = [cell for cell in neighborhood if not self.model.grid.get_cell_list_contents(cell)] # Ver celdas disponibles
        if open_cells:  # Si hay celdas disponibles
            new_position = self.random.choice(open_cells) # Posision aleatoria dentro de las disponibles
            self.model.grid.move_agent(self, new_position) # Mover al agente

    # Funcion step que hay que definir
    def step(self):
        if self.model.is_dirt(self.pos): # Si la celda actual tiene basura
            self.model.clean_dirt(self.pos) # Limpiar
            self.move() # Mover
        else:
            self.move() # Mover

In [810]:
class CleaningModel(Model):
    # Constructor del Modelo
    def __init__(self, width, height, num_agents, dirty_cells, max_steps):
        self.num_agents = num_agents # Numero de Agentes
        self.grid = MultiGrid(width, height, torus=False) # Espacio donde van a vivir los agentes
        self.schedule = RandomActivation(self) # Activar de manera aleatoria 
        self.max_steps = max_steps # Numero maximo de pasos a ejecutar
        self.steps = 0 # Contador de pasos
        self.dirt = np.zeros((width, height)) # Arreglo 2D de 0 para iniciar basura

        # Inicializar basura
        num_dirty_cells = int(width * height * dirty_cells) # Cantidad de basura
        all_cells = [(x, y) for x in range(width) for y in range(height)] # Lista con coordenadas disponibles        
        random.shuffle(all_cells) # Mezclar aleatoriamente las coordenadas disponibles
        dirty_cells = all_cells[:num_dirty_cells] # Guardar rango de # de basuras
        for cell in dirty_cells:
            self.dirt[cell] = 1 # Guardar basura

        # Inicializar Agentes
        for i in range(num_agents):
            agent = CleaningAgent(i, self) # Crear Objeto
            self.grid.place_agent(agent, (1, 1)) # Ponerlo en el grid en la posicion (1, 1)
            self.schedule.add(agent) # Agregarloal schedule

    # Funcion para checar si es basura
    def is_dirt(self, pos):
        return self.dirt[pos[0]][pos[1]] == 1

    # Funcion para limpiar
    def clean_dirt(self, pos):
        self.dirt[pos[0]][pos[1]] = 0
    
    # Funcion step
    def step(self):
        self.schedule.step()
        self.steps += 1 # Aumentar step


In [811]:
# Funcion para correr la simulacion
def run_cleaning_simulation(width, height, num_agents, dirty_cells, max_steps):
    model = CleaningModel(width, height, num_agents, dirty_cells, max_steps) # Crear objeto modelo
    for step in range(max_steps):
        model.step()
        if np.all(model.dirt == 0): # Si model.grid.dirt son puros 0
            return (
                step, # Numero de step
                (1 - np.mean(model.dirt)) * 100 # Porcentaje de limpios
            )
    # Si se corren todos los steps y aun no se limpia todo
    return (
        max_steps, 
        (1 - np.mean(model.dirt)) * 100 
    )

In [812]:
# CONSTANTES
WIDTH = 100
HEIGHT = 100
NUM_AGENTS = 10
DIRTY_CELLS = .9
MAX_STEPS = 100

start_time = time.time() # Empezar timer
result = run_cleaning_simulation(WIDTH, HEIGHT, NUM_AGENTS, DIRTY_CELLS, MAX_STEPS) # Llamar funcion
total_time =  time.time() - start_time # Detener timer

print("The simulation ran in: " + str(total_time))
print("The simulation took: " + str(result[0]) + " steps")
print("Percentage of clean cells: " + str(result[1]) + "%")

The simulation ran in: 0.04798483848571777
The simulation took: 100 steps
Percentage of clean cells: 11.6%
